# Reinforcement Learning: Policy Gradients

Homework assignments are mandatory. In order to be granted with 8 ECTS and your grade, you must pass 9 out of 10 homeworks. Upload your solution as zip archive until Saturday 13th January 23:59 into the public Homework Submissions folder on studip. The archive should contain your solution as iPython notebook (.ipynb) and as HTML export. Name the archive ﬁle as <your group id > policy-gradients.zip.

Further, you have to correct another group’s homework until Monday 15th January 23:59. Please follow the instructions in the rating guidelines on how to do your rating.

If you encounter problems, please do not hesitate to send your question or concern to lbraun@uos.de. Please do not forget to include [TF] in the subject line.

# 1 Introduction

In this week’s task, we are going to implement a network that learns to balance a pole on a cart. The network will learn how to map an observation to an appropriate action.
# 2 Data, OpenAI Gym
Install the OpenAI Gym pip package (pip instsall gym). Create a CartPolev0 environment and create a loop that can retrieve an observation and selects and executes a random action, until the episode is done. Create a loop around that loop, such that you can run the environment for several episodes. Render the environment every N episodes and keep in mind, that rendering does slow down the learning drastically, since if the process is not visualized, it does not restrict the frame rate and runs as fast as possible.

Observation 

Each observation is a four-dimensional array: [cart position, cart velocity, pole angle, pole velocity at tip].

Action 

space At each time-step, the agent has to decide to either push the cart to the left or the right.

Reward 

The agent receives a +1 reward for each time-step until the episode terminates.

Episode Termination

The episode terminates, if either the pole angle is more than +/- 12 degree, the cart moved out of the display or if the episode length is greater than 200. Hence, the maximum reward that can be reached is 200.

In [18]:
import gym
import numpy as np
import tensorflow as  tf

#### the graph ####
x = tf.placeholder(tf.float32, shape = (1,4))
# hidden layer
w01 = tf.Variable(tf.random_normal(shape = (4,8), stddev = 0.1))
b01 = tf.Variable(tf.random_normal(shape = (1,8), stddev  = 0.1))
l01 = tf.nn.relu(tf.add(tf.matmul(x,w01), b01))

# output layer
w02 = tf.Variable(tf.random_normal(shape = (8,1), stddev = 0.1))
b02 = tf.Variable(tf.random_normal(shape = (1,1), stddev  = 0.1))
prob01 = tf.nn.sigmoid(tf.add(tf.matmul(l01,w02), b02))
prob02 = tf.subtract(1.0, prob01)

# Create log likelihoods from a probability distribution over actions
log_likelihoods = tf.log(tf.concat([prob01, prob02],1))

# sample action from that distribution
action = tf.multinomial(log_likelihoods,1)[0][0]
print(action)

# select value that corresponds to selected action
log_likelihood = log_likelihoods[:, tf.to_int32(action)]
print(log_likelihood)

# calculate gradient with respect to that value

####parameters####
episodes = 100
episode_length = 200
learn_rate = 0.1

####execution####
with tf.Session() as session:
    
    # initialize variables
    session.run(tf.global_variables_initializer())
    
    # create environment
    env = gym.make('CartPole-v0')
    
    
    for episode in np.arange(episodes):
        
        # Reset Game
        observation = env.reset()
        
        for step in np.arange(episode_length):
            
            observation = observation[np.newaxis]
            #print(observation)
            
            # Create an action (i.e. sample from network)
            _action = session.run(action, feed_dict = {x: observation})
            
            # Execute action and receive the corresponding reward 
            # and a new observation
            observation, reward, done, _ = env.step(_action)
            
            
            if done:
                break
                
            # render the visualization
            #if episodes%10:
                #env.render()

Tensor("strided_slice_39:0", shape=(), dtype=int64)
Tensor("strided_slice_40:0", shape=(1,), dtype=float32)


In [49]:
import tensorflow as tf

#### the graph ####
x = tf.placeholder(tf.float32, shape = (1,4))

# hidden layer
w01 = tf.Variable(tf.random_normal(shape = (4,8), stddev = 0.1))
b01 = tf.Variable(tf.random_normal(shape = (1,8), stddev = 0.1))
l01 = tf.nn.relu(tf.add(tf.matmul(x,w01), b01))

# ouput layer
w02 = tf.Variable(tf.random_normal(shape = (8,1), stddev = 0.1))
b02 = tf.Variable(tf.random_normal(shape = (1,1), stddev = 0.1))
p_right = tf.nn.relu(tf.add(tf.matmul(l01,w02), b02))
p_left = tf.subtract(1.0,p_right)

In [50]:
# draw and 
# create log likelihoods from a probability distribution over actions
action = tf.multinomial(tf.log(tf.concat([p_right, p_left], 1)), 1)[0][0]
print(action)

####parameters####
epochs = 100
episode_length = 200
learn_rate = 0.1

Tensor("strided_slice_9:0", shape=(), dtype=int64)


In [51]:
import gym
import numpy as np

# Create new environment instance 
env = gym.make("CartPole-v0")

# Investigate environment 
print(env.metadata) 
print(env.observation_space) 
print(env.action_space)

# Sample a random action 
action = env.action_space.sample() 
print(action)

with tf.Session() as session:
    
    # initialize
    session.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        # Reset Game
        observation = env.reset()
        observation = observation[np.newaxis]
        #observation = (env.reset())[np.newaxis]
        #observation = np.concatenate((np.array([[1]]),[env.reset()]), 1)
        #print(observation)
        #print(observation.shape)
        done = False

        while not done:
            # Render current game state (slows down learning!)
            #if epoch%5:
                #env.render()

            # Create an action (i.e. sample from network)
            observation, reward, done, info = env.step(session.run(action, feed_dict = {x: observation}))

            #env.step(session.run(action, feed_dict = {x:observation}))
            
            action = env.action_space.sample()
            
            # Execute action and receive the corresponding reward 
            # and a new observation
            observation, reward, done, info = env.step(action)

        # Improve network ...


{'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 50}
Box(4,)
Discrete(2)
1


TypeError: Fetch argument 1 has invalid type <class 'int'>, must be a string or Tensor. (Can not convert a int into a Tensor or Operation.)